In [1]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import json
import pandas as pd
import re
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yaweiqin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yaweiqin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
raw_data = pd.read_json('train.json')

In [3]:
raw_data.describe()

/var/folders/tw/s57yx3rx6_501ddjjznfldvh0000gn/T/ipykernel_83664/3646701449.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  raw_data.describe()


,abstract,date,subject,title
count,156543,156543,156543,156543
unique,156412,7258,30,156425
top,This paper has been withdrawn.,2012-10-04 00:00:00+00:00,mesoscale and nanoscale physics,Crossover from Isotropic to Directed Percolation
freq,18,78,23440,3
first,NaN,1992-03-28 00:00:00+00:00,NaN,NaN
last,NaN,2012-12-31 00:00:00+00:00,NaN,NaN


In [4]:
withrawn_drop = raw_data.loc[(raw_data['abstract'].str.contains('withdrawn')) & \
			 (raw_data['abstract'].str.contains('paper')) |
			 (raw_data['abstract'].str.contains('withdrawn')) & \
			 (raw_data['abstract'].str.contains('submission'))
			].index 
train_data_cleaned = raw_data.drop(withrawn_drop, axis=0)
train_data_cleaned.describe()

/var/folders/tw/s57yx3rx6_501ddjjznfldvh0000gn/T/ipykernel_83664/3914242922.py:7: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  train_data_cleaned.describe()


,abstract,date,subject,title
count,156272,156272,156272,156272
unique,156199,7257,30,156160
top,Quantum criticality is the intriguing possib...,2012-10-04 00:00:00+00:00,mesoscale and nanoscale physics,Quantum phase transitions
freq,2,78,23407,3
first,NaN,1992-03-28 00:00:00+00:00,NaN,NaN
last,NaN,2012-12-31 00:00:00+00:00,NaN,NaN


 clean
 1. normalize, lower case text = text.lower()
 2. Removing Unicode Characters,
 import re
 text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
 3.Removing Stopwords
 import nltk.corpus
 nltk.download('stopwords')
 from nltk.corpus import stopwords
 stop = stopwords.words('english')
 text = " ".join(\[word for word in text.split() if word not in (stop)])
 4.Stemming and Lemmatization
 import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

words = \["jump", "jumped", "jumps", "jumping"]
stemmer = PorterStemmer()
for word in words:

print(word + " = " + stemmer.stem(word))
lemmatizer = WordNetLemmatizer()
for word in words:

print(word + " = " + lemmatizer.lemmatize(word))

In [5]:
stop = stopwords.words('english')
#stemmer = PorterStemmer()
#lemmatizer = WordNetLemmatizer()
def data_clean(text):
	text = re.sub(r"[^\w\s]", " ", str(text).lower().strip()).split()
	text = [word for word in text if word not in (stop)]
	#text = [stemmer.stem(word) for word in text]
	#text = [lemmatizer.lemmatize(word) for word in text]
	text = ' '.join(text)
	return(text)

In [6]:
init_time = time.time()
train_data_cleaned['cleaned_abstract'] = train_data_cleaned['abstract'].apply(data_clean) #lambda x: data_clean(x))
mid_time = time.time() 
print(mid_time - init_time)
train_data_cleaned['cleaned_title'] = train_data_cleaned['title'].apply(data_clean) #lambda x: data_clean(x))
print(time.time() - mid_time)

34.67321991920471
3.675818920135498


In [7]:
train_data_cleaned['combined'] = train_data_cleaned['cleaned_abstract'] + train_data_cleaned['title']
train_data_cleaned.head()

,abstract,date,subject,title,cleaned_abstract,cleaned_title,combined
0,We study the two-particle wave function of p...,2007-03-31 00:00:00+00:00,mesoscale and nanoscale physics,Bosonic characters of atomic Cooper pairs acro...,study two particle wave function paired atoms ...,bosonic characters atomic cooper pairs across ...,study two particle wave function paired atoms ...
1,A general formulation was developed to repre...,2007-03-31 00:00:00+00:00,materials science,Numerical solution of shock and ramp compressi...,general formulation developed represent materi...,numerical solution shock ramp compression gene...,general formulation developed represent materi...
2,We present recent advances in understanding ...,2007-04-02 00:00:00+00:00,strongly correlated electrons,Spectroscopic Properties of Polarons in Strong...,present recent advances understanding ground e...,spectroscopic properties polarons strongly cor...,present recent advances understanding ground e...
3,We describe a peculiar fine structure acquir...,2007-04-02 00:00:00+00:00,mesoscale and nanoscale physics,Filling-Factor-Dependent Magnetophonon Resonan...,describe peculiar fine structure acquired plan...,filling factor dependent magnetophonon resonan...,describe peculiar fine structure acquired plan...
4,We investigate the effect of tuning the phon...,2007-03-31 00:00:00+00:00,strongly correlated electrons,Tuning correlation effects with electron-phono...,investigate effect tuning phonon energy correl...,tuning correlation effects electron phonon int...,investigate effect tuning phonon energy correl...


In [8]:
random_seed = 13
train_data, test_data, train_target, test_target = train_test_split(train_data_cleaned['combined'], 
                                                    train_data_cleaned['subject'], 
                                                    test_size=1/3, 
                                                    random_state=random_seed)

In [9]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=random_seed)),])
					# LogisticRegression() MultinomialNB()
_ = text_clf_svm.fit(train_data, train_target)
predicted_svm = text_clf_svm.predict(test_data)
np.mean(predicted_svm == test_target)

0.5791019561920485